In [97]:
import cv2
import numpy as np

# Define the known size of the ArUco marker (e.g., in meters or any other unit)
# This size should be the side length of the marker
marker_length = 0.02  


# Read the video
cap = cv2.VideoCapture('aruco.mp4')

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Assume the principal point is at the center of the image
cx = frame_width / 2
cy = frame_height / 2

# Prepare the ArUco detection
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

# Placeholder for focal length estimation
focal_length_estimations = []

# Process the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # If markers are detected
    if ids is not None:
        for corner in corners:
            # Estimate focal length based on the known marker size
            # This is done using the formula: (pixel_size / marker_size_in_pixels) * marker_size = focal_length
            marker_size_in_pixels = cv2.norm(corner[0][0] - corner[0][1])
            focal_length_estimate = (marker_size_in_pixels / marker_length) * marker_length
            focal_length_estimations.append(focal_length_estimate)

if focal_length_estimations:
    # Average the focal length estimations
    estimated_focal_length = np.mean(focal_length_estimations)
else:
    # Fallback if no markers were detected
    estimated_focal_length = max(frame_width, frame_height)

# Assuming zero distortion coefficients
distCoeffs = np.zeros(5)

# Construct the estimated camera matrix
cameraMatrix = np.array([[estimated_focal_length, 0, cx],
                         [0, estimated_focal_length, cy],
                         [0, 0, 1]], dtype='float64')

print("Estimated Camera Matrix:\n", cameraMatrix)
print("Assumed Distortion Coefficients:\n", distCoeffs)

# Release the video capture
cap.release()

Estimated Camera Matrix:
 [[128.99755994   0.         960.        ]
 [  0.         128.99755994 540.        ]
 [  0.           0.           1.        ]]
Assumed Distortion Coefficients:
 [0. 0. 0. 0. 0.]


In [13]:
import cv2
import cv2.aruco as aruco
import numpy as np

# Load the ArUco dictionary and create parameters for detection
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

# Define the camera matrix and distortion coefficients
camera_matrix = np.array([[128.99755994, 0, 960], [0, 128.99755994, 540], [0, 0, 1]], dtype=np.float32)
dist_coeffs = np.zeros((5, 1), dtype=np.float32)  # Assuming no distortion

# Define the 3D coordinates of the cube vertices
cube_size = 0.05 
half_cube_size = cube_size / 2
cube_height = -cube_size / 5  

cube_points_3d = np.float32([
    [-half_cube_size, half_cube_size, 0], [half_cube_size, half_cube_size, 0],
    [half_cube_size, -half_cube_size, 0], [-half_cube_size, -half_cube_size, 0],
    [-half_cube_size, half_cube_size, -cube_height], [half_cube_size, half_cube_size, -cube_height],
    [half_cube_size, -half_cube_size, -cube_height], [-half_cube_size, -half_cube_size, -cube_height]
])

# Edges of the cube
edges = [
    (0, 1), (1, 2), (2, 3), (3, 0),  # Bottom square
    (4, 5), (5, 6), (6, 7), (7, 4),  # Top square
    (0, 4), (1, 5), (2, 6), (3, 7)   # Pillars
]

colors = [
    (0, 0, 255),  # Blue
    (0, 255, 0),  # Green
    (255, 0, 0),  # Red
    (0, 255, 255),  # Cyan
    (255, 0, 255),  # Magenta
    (255, 255, 0),  # Yellow
]

# Initialize video capture and the video writer
cap = cv2.VideoCapture('aruco.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Store the last known good transform for marker ID2
last_valid_transform_ID2 = None
# Threshold for detecting jumps in translation for ID2 (tune as necessary)
transformation_threshold = 0.05


# Draw colored cube function
def draw_colored_cube(img, corners, colors):
    imgpts = np.int32(corners).reshape(-1, 2)  # Ensure the points are integer type

    # Bottom face
    cv2.fillConvexPoly(img, imgpts[[0, 1, 2, 3]], colors[0])

    # Top face
    cv2.fillConvexPoly(img, imgpts[[4, 5, 6, 7]], colors[1])

    # Front face
    cv2.fillConvexPoly(img, imgpts[[0, 1, 5, 4]], colors[2])

    # Back face
    cv2.fillConvexPoly(img, imgpts[[2, 3, 7, 6]], colors[3])

    # Left face
    cv2.fillConvexPoly(img, imgpts[[0, 3, 7, 4]], colors[4])

    # Right face
    cv2.fillConvexPoly(img, imgpts[[1, 2, 6, 5]], colors[5])

    # Optionally, you can draw the edges as well for better visual distinction
    for edge in edges:
        pt1 = tuple(imgpts[edge[0]])
        pt2 = tuple(imgpts[edge[1]])
        cv2.line(img, pt1, pt2, (0, 0, 0), 1, cv2.LINE_AA)

    return img

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    corners, ids, rejected = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # If markers are detected
    if ids is not None:
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, cube_size, camera_matrix, dist_coeffs)

        for rvec, tvec, marker_id in zip(rvecs, tvecs, ids.flatten()):
        
            # Project 3D points to image plane
            imgpts, _ = cv2.projectPoints(cube_points_3d, rvec, tvec, camera_matrix, dist_coeffs)

            # Draw the solid cube
            frame = draw_colored_cube(frame, imgpts, colors)

    # Write the modified frame to the output video file
    out.write(frame)

# Release video capture and video write objects
cap.release()
out.release()
cv2.destroyAllWindows()